In [1]:
# Libraries
import pandas as pd

# Load up store_income_data_task.csv
income_df = pd.read_table('store_income_data_task.csv', sep = ',')

In [2]:
# Function for display unique values
def display_unique_values(income_df, column_name):
    # Exclude empty string values
    unique_values = income_df[column_name][income_df[column_name] != ''].unique()
    print(f"There are {len(unique_values)} unique values in the '{column_name}' column.")
    print(f"Unique values:\n{unique_values}")

In [3]:
# Display unique countries information (Exclude empty string values)
display_unique_values(income_df, 'country')

There are 77 unique values in the 'country' column.
Unique values:
['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' nan 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' ' ' 'UK.' 'England/' 'england'
 ' Britain' 'united states of america' 'UK/' 'SA/' 'SA' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' ' U.K'
 ' United States of America' 'Britain ' 'England' ' SA'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. Africa

In [4]:
# Convert to lower case
income_df['country'] = income_df['country'].str.lower()

# Remove trailing white spaces
income_df['country'] = income_df['country'].str.strip()

# Remove all symbols
income_df['country'] = income_df['country'].str.replace(r'[^\w\s]', '', regex=True)

# Replace NaN values with empty strings
income_df['country'] = income_df['country'].fillna("")

# Display unique countries information (Exclude empty string values)
display_unique_values(income_df, 'country')

There are 9 unique values in the 'country' column.
Unique values:
['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' 'england'
 'united states of america' 's africasouth africa']


In [5]:
# Backup country column for verification later 
income_df['country_original'] = income_df['country']

# Display unique values before cleaning
income_df[['country', 'country_original']]
grouped = income_df.groupby(['country', 'country_original']).size().reset_index(name='count')

print('Display unique values before cleaning')
print(grouped)

Display unique values before cleaning
                    country          country_original  count
0                                                         86
1                   america                   america     75
2                   britain                   britain     95
3                   england                   england     76
4      s africasouth africa      s africasouth africa     76
5                        sa                        sa    184
6                        uk                        uk    170
7            united kingdom            united kingdom     90
8             united states             united states     81
9  united states of america  united states of america     67


In [6]:
# Display unique values in descending order of occurence
grouped = income_df.groupby(['country', 'country_original']).size().reset_index(name='count')
sorted_grouped = grouped.sort_values(by='count', ascending=False)

print(sorted_grouped)

                    country          country_original  count
5                        sa                        sa    184
6                        uk                        uk    170
2                   britain                   britain     95
7            united kingdom            united kingdom     90
0                                                         86
8             united states             united states     81
4      s africasouth africa      s africasouth africa     76
3                   england                   england     76
1                   america                   america     75
9  united states of america  united states of america     67


In [7]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
from fuzzywuzzy import process

# Define a function to replace similar values
def replace_similar_values(income_df, column, target_value, threshold=90):
    unique_values = income_df[column].unique()
    matches = process.extract(target_value, unique_values, limit=None)
    similar_values = [match[0] for match in matches if match[1] >= threshold]
    income_df[column] = income_df[column].replace(similar_values, target_value)

# Apply the function to consolidate country names
replace_similar_values(income_df, 'country', 'united states')
replace_similar_values(income_df, 'country', 'south africa')

# Verify country with the backup
income_df[['country', 'country_original']]
grouped = income_df.groupby(['country', 'country_original']).size().reset_index(name='count')

print('Display unique values after fuzzywuzzy cleaning')
print(grouped)


Display unique values after fuzzywuzzy cleaning
          country          country_original  count
0                                               86
1         america                   america     75
2         britain                   britain     95
3         england                   england     76
4              sa                        sa    184
5    south africa      s africasouth africa     76
6              uk                        uk    170
7  united kingdom            united kingdom     90
8   united states             united states     81
9   united states  united states of america     67


In [8]:
# Define a mapping of old values to new values
mapping = {
    'uk':                   'united kingdom',
    'britain':              'united kingdom',
    'england':              'united kingdom',
    'united kingdom':       'united kingdom',
	'sa':                   'south africa',
	'united states':        'united states of america',
	'america':              'united states of america'
}

# Apply the mapping to the 'country' column
income_df['country'] = income_df['country'].replace(mapping)

# Verify the result
print(income_df['country'].value_counts())

country
united kingdom              431
south africa                260
united states of america    223
                             86
Name: count, dtype: int64


In [9]:
# Verify country with the backup
income_df[['country', 'country_original']]
grouped = income_df.groupby(['country', 'country_original']).size().reset_index(name='count')

print('Display unique values after mapping cleaning for eyeball check')
print(grouped)

Display unique values after mapping cleaning for eyeball check
                    country          country_original  count
0                                                         86
1              south africa      s africasouth africa     76
2              south africa                        sa    184
3            united kingdom                   britain     95
4            united kingdom                   england     76
5            united kingdom                        uk    170
6            united kingdom            united kingdom     90
7  united states of america                   america     75
8  united states of america             united states     81
9  united states of america  united states of america     67


In [10]:
# After verification of cleaned country, drop column country_original
income_df.drop(columns=['country_original'], inplace=True)

In [11]:
# Display unique countries information
display_unique_values(income_df, 'country')

There are 3 unique values in the 'country' column.
Unique values:
['united states of america' 'united kingdom' 'south africa']


In [12]:
# Import modules
from datetime import date

# Create column, days_ago, by using current date minus parsed date_measured
income_df['days_ago'] = (pd.to_datetime(date.today()) - pd.to_datetime(income_df['date_measured'], dayfirst=True, errors='coerce')).dt.days

# Identify rows with NaT entries
err_entries = income_df[income_df['days_ago'].isna()]

# Reminder message for data analyst
if len(err_entries) > 0:
    print(f"No. of NaT entries: {len(err_entries)};  REMEMBER TO FOLLOW UP!")
    # Display the rows with NaT entries
    print(err_entries[['id','days_ago','date_measured']])
else:
    print("There is no NaT entries after parsing date_measured, i.e. date parsing is successful for all records.")

There is no NaT entries after parsing date_measured, i.e. date parsing is successful for all records.


In [13]:
# Display cleaned data of the first 20 records
income_df.head(20)

,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states of america,6925
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united kingdom,6956
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states of america,7801
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united kingdom,6832
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom,18992
5,6,"Auburn National Bancorporation, Inc.",ccaldeyroux5@dion.ne.jp,Grocery,$69798987.04,19-9-1999,united kingdom,9255
6,7,"Interlink Electronics, Inc.",orodenborch6@skyrock.com,Garden,$22521052.79,8-6-2001,south africa,8627
7,8,"Tallgrass Energy Partners, LP",NaN,Grocery,$54405380.40,16-9-1992,united kingdom,11814
8,9,Tronox Limited,NaN,Outdoors,$99290004.13,11-1-1992,united states of america,12063
9,10,"Synopsys, Inc.",lcancellieri9@tmall.com,Electronics,$44091294.62,11-7-2006,united kingdom,6768
